# 1.Omniverse Isaac Sim Glossary术语表

[Glossary术语表](https://docs.omniverse.nvidia.com/isaacsim/latest/reference_glossary.html#isaac-sim-glossary-prim)


# 2.Omnigraph

#### 2.1
[Action Graph Quickstart](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/tutorials/quickstart.html)
- 一个类似blender的例子。
  - First, load the `Action Graph Bundle extension` into Omniverse USD Composer
  - Use a Write Prim Attribute Node: **Drag** the prim from the Stage to the Action Graph editor
  - flipflop node

[OmniGraph](https://docs.omniverse.nvidia.com/isaacsim/latest/gui_tutorials/tutorial_gui_omnigraph.html#)

- jetbot法1，手动链接graph node
- 两个外设输入例子：
Isaac Examples > Input Devices > Kaya Gamepad/Omnigraph Keyboard，在stage中xx_graph上右键单击**Open Graph**可以查看编辑

#### 2.2
进一步的封装常用功能

[Commonly Used Omnigraph Shortcuts](https://docs.omniverse.nvidia.com/isaacsim/latest/advanced_tutorials/tutorial_advanced_omnigraph_shortcuts.html#commonly-used-omnigraph-shortcuts)

jetbot法2【articulation controller】【differential Controller】

#### 2.3
Prerequisites，先看看# 4.Interactive Scripting 交互式脚本

[scripting OmniGraph via Python](https://docs.omniverse.nvidia.com/isaacsim/latest/advanced_tutorials/tutorial_advanced_omnigraph_scripting.html#isaac-sim-app-tutorial-advanced-omnigraph-scripting)

In [ ]:
import omni.graph.core as og

keys = og.Controller.Keys
(graph_handle, list_of_nodes, _, _) = og.Controller.edit(
    {"graph_path": "/action_graph", "evaluator_name": "execution"},
    {
        keys.CREATE_NODES: [
            ("tick", "omni.graph.action.OnTick"),
            ("print","omni.graph.ui_nodes.PrintText")
        ],
        keys.SET_VALUES: [
            ("print.inputs:text", "Hello World"),
            ("print.inputs:logLevel","Warning")                 # setting the log level to warning so we can see the printout in terminal
        ],
        keys.CONNECT: [
            ("tick.outputs:tick", "print.inputs:execIn")
        ],
    },
)

In [ ]:
# get existing value from an attribute
existing_text = og.Controller.attribute("/action_graph/print.inputs:text").get()
print("Existing Text: ", existing_text)

# set new value
og.Controller.attribute("/action_graph/print.inputs:text").set("New Texts to print")

In [ ]:
og.Controller.create_node("/action_graph/new_node_name", "omni.graph.nodes.ConstantString")
og.Controller.attribute("/action_graph/new_node_name.inputs:value").set("This is a new node")
og.Controller.connect("/action_graph/new_node_name.inputs:value", "/action_graph/print.inputs:text")

有一个特殊参数称为“pipeline_stage”，可以将Graph设置为“按需”执行，仅在调用时执行来显式触发而不是在仿真的每一帧上进行。

通常，在创建图形时设置此变量，如下所示`"pipeline_stage": og.GraphPipelineStage.GRAPH_PIPELINE_STAGE_ONDEMAND`。

In [ ]:
(demand_graph_handle,_,_,_) = og.Controller.edit(
    {
        "graph_path": "/ondemand_graph",
        "evaluator_name": "execution",
        "pipeline_stage": og.GraphPipelineStage.GRAPH_PIPELINE_STAGE_ONDEMAND
    },
    {
        keys.CREATE_NODES: [
            ("tick", "omni.graph.action.OnTick"),
            ("print","omni.graph.ui_nodes.PrintText")
        ],
        keys.SET_VALUES: [
            ("print.inputs:text", "On Demand Graph"),
            ("print.inputs:logLevel","Warning")
        ],
        keys.CONNECT: [
            ("tick.outputs:tick", "print.inputs:execIn")
        ],
    },
)

或者，您也可以通过以下方式为已经创建的Graph设置它：`yourGraph_handle.change_pipeline_stage(og.GraphPipelineStage.GRAPH_PIPELINE_STAGE_ONDEMAND)`

测试：
Start simulation by press “play”. 
To manually trigger a graph, open another tab in the Script Editor, and paste in`yourGraph_handle.evaluate()`

> 有关将图形附加到物理回调和/或渲染回调的更深入示例，请参见`standalone_examples/api/omni.isaac.core/omnigraph_triggers.py`

#### 2.4

Omnigraph’s Custom Node Tutorial

[Part One](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/tutorials/deforming_text_run_graph.html)



# 3.Attach a Camera to Robot

[Attach a Camera to Robot](https://docs.omniverse.nvidia.com/isaacsim/latest/gui_tutorials/tutorial_gui_camera_sensors.html#attach-a-camera-to-robot)

# 4.Interactive Scripting 交互式脚本

1. Script Editor 内部的脚本编辑器 
   
   通过在脚本编辑器下的标签菜单中打开多个标签。所有标签共享相同的环境，因此在一个环境中导入的库或定义的变量可以在其他环境中访问和使用。
2. REPL Extension
   
   启用Isaac Sim REPL扩展，打开一个新的终端，在命令行中运行 `telnet localhost 8223`。终端中会启动一个Python shell，
   这样就可以在shell里通过 Python 与在 Isaac Sim 中打开的Stage进行交互。
3. Raw USD APIs
   > 注意：Raw USD APIs are versatile and detailed but complex, especially for beginners. 
4. Isaac Sim Core APIs !
   > Isaac Sim has a set of core APIs that simplify some of the frequently used actions for robotics simulators. These APIs abstract away default parameter settings.
   > 
   > Using Isaac Sim Core APIs produces code that is lightweight and readable. When necessary you can always use USD APIs to direct the stage.
